# 04: Model Zoo - Choosing the Right Model

**Course:** 21CSE558T - Deep Neural Network Architectures  
**Module 4:** CNNs & Transfer Learning (Week 12)  
**Estimated Time:** 10-12 minutes  
**Prerequisites:** Notebooks 01-03  
**Goal:** Compare VGG16, ResNet50, MobileNetV2

---

## 📚 What You'll Learn

In this notebook, you will:
1. Compare three popular pre-trained models
2. Understand size vs accuracy tradeoffs
3. Learn when to use each model
4. Make informed model selection decisions

**Key Question:** _"Which model should I use for MY project?"_

---

In [ ]:
# Setup
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import time
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

print(f"TensorFlow version: {tf.__version__}")
print("Ready to compare models!\n")

## Step 1: Load Dataset

In [ ]:
# Load dataset
(train_ds, val_ds), info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,
    with_info=True
)

num_classes = info.features['label'].num_classes
class_names = info.features['label'].names

# Preprocess
IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    # Use model-specific preprocessing
    # For simplicity, we'll use ResNet50's preprocessing for all models
    image = tf.keras.applications.resnet50.preprocess_input(image)
    return image, label

train_ds = train_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Dataset ready!")

## Step 2: Meet the Contenders

### 🦁 VGG16 - The Simple Giant
- **Year:** 2014
- **Layers:** 16
- **Architecture:** Simple stacked Conv layers
- **Parameters:** 138M
- **Size:** 528 MB
- **ImageNet Accuracy:** 71.3%

**Pros:** Simple architecture, easy to understand, good for learning
**Cons:** HUGE model size, slow training, outdated

---

### 🚀 ResNet50 - The Default Choice ⭐
- **Year:** 2015
- **Layers:** 50
- **Architecture:** Residual connections (skip connections)
- **Parameters:** 25.6M
- **Size:** 98 MB
- **ImageNet Accuracy:** 76.1%

**Pros:** Excellent accuracy, reasonable size, widely used, default choice!
**Cons:** Moderate speed

---

### 📱 MobileNetV2 - The Speed Demon
- **Year:** 2018
- **Layers:** 53
- **Architecture:** Depthwise separable convolutions
- **Parameters:** 3.5M
- **Size:** 14 MB
- **ImageNet Accuracy:** 71.8%

**Pros:** TINY size, FAST inference, perfect for mobile!
**Cons:** Slightly lower accuracy than ResNet50

---

## Step 3: Load All Three Models

In [ ]:
# Load models
print("Loading VGG16...")
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
vgg16_base.trainable = False

print("Loading ResNet50...")
resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
resnet50_base.trainable = False

print("Loading MobileNetV2...")
mobilenet_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
mobilenet_base.trainable = False

print("\n✅ All models loaded!\n")

# Compare parameters
print("=" * 60)
print(f"{'Model':<20} {'Parameters':<20} {'Size (approx)'}")
print("=" * 60)
print(f"{'VGG16':<20} {vgg16_base.count_params():>15,}   {'528 MB'}")
print(f"{'ResNet50':<20} {resnet50_base.count_params():>15,}   {'98 MB'}")
print(f"{'MobileNetV2':<20} {mobilenet_base.count_params():>15,}   {'14 MB'}")
print("=" * 60)
print("\nMobileNetV2 is 10× smaller than ResNet50!")
print("VGG16 is 5× larger than ResNet50!")

## Step 4: Build Models with Feature Extraction

In [ ]:
# Build VGG16 model
vgg16_model = Sequential([
    vgg16_base,
    GlobalAveragePooling2D(),
    Dense(num_classes, activation='softmax')
], name='VGG16_Model')

vgg16_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Build ResNet50 model
resnet50_model = Sequential([
    resnet50_base,
    GlobalAveragePooling2D(),
    Dense(num_classes, activation='softmax')
], name='ResNet50_Model')

resnet50_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Build MobileNetV2 model
mobilenet_model = Sequential([
    mobilenet_base,
    GlobalAveragePooling2D(),
    Dense(num_classes, activation='softmax')
], name='MobileNetV2_Model')

mobilenet_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ All 3 models built and compiled!")

## Step 5: Train All Three Models

We'll train each model for 5 epochs and compare:
- Training time
- Validation accuracy
- Memory usage

In [ ]:
# Train VGG16
print("\n" + "="*60)
print("Training VGG16...")
print("="*60)
start_time = time.time()
vgg16_history = vgg16_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    verbose=1
)
vgg16_time = time.time() - start_time
print(f"\n✅ VGG16 training time: {vgg16_time:.1f} seconds")

In [ ]:
# Train ResNet50
print("\n" + "="*60)
print("Training ResNet50...")
print("="*60)
start_time = time.time()
resnet50_history = resnet50_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    verbose=1
)
resnet50_time = time.time() - start_time
print(f"\n✅ ResNet50 training time: {resnet50_time:.1f} seconds")

In [ ]:
# Train MobileNetV2
print("\n" + "="*60)
print("Training MobileNetV2...")
print("="*60)
start_time = time.time()
mobilenet_history = mobilenet_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    verbose=1
)
mobilenet_time = time.time() - start_time
print(f"\n✅ MobileNetV2 training time: {mobilenet_time:.1f} seconds")

## Step 6: Compare Results

In [ ]:
# Extract final accuracies
vgg16_acc = vgg16_history.history['val_accuracy'][-1]
resnet50_acc = resnet50_history.history['val_accuracy'][-1]
mobilenet_acc = mobilenet_history.history['val_accuracy'][-1]

# Print comparison table
print("\n" + "="*80)
print("🏆 FINAL COMPARISON - MODEL ZOO")
print("="*80)
print(f"\n{'Model':<15} {'Params':<15} {'Size':<12} {'Time':<12} {'Val Acc':<12} {'Best For'}")
print("-"*80)
print(f"{'VGG16':<15} {'138M':<15} {'528 MB':<12} {f'{vgg16_time:.0f}s':<12} {f'{vgg16_acc:.1%}':<12} {'Learning'}")
print(f"{'ResNet50 ⭐':<15} {'26M':<15} {'98 MB':<12} {f'{resnet50_time:.0f}s':<12} {f'{resnet50_acc:.1%}':<12} {'Default!'}")
print(f"{'MobileNetV2':<15} {'3.5M':<15} {'14 MB':<12} {f'{mobilenet_time:.0f}s':<12} {f'{mobilenet_acc:.1%}':<12} {'Mobile'}")
print("-"*80)
print("\n✨ ResNet50 offers best balance of accuracy, size, and speed!")
print("📱 MobileNetV2 is 7× faster and 10× smaller!")
print("📚 VGG16 is good for learning but outdated for production")
print("="*80)

## Step 7: Visualize Comparison

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Validation Accuracy Comparison
ax1 = axes[0, 0]
ax1.plot(vgg16_history.history['val_accuracy'], label='VGG16', marker='o')
ax1.plot(resnet50_history.history['val_accuracy'], label='ResNet50 ⭐', marker='s', linewidth=2)
ax1.plot(mobilenet_history.history['val_accuracy'], label='MobileNetV2', marker='^')
ax1.set_title('Validation Accuracy', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Model Size Comparison
ax2 = axes[0, 1]
models = ['VGG16', 'ResNet50', 'MobileNetV2']
sizes = [528, 98, 14]  # MB
colors = ['red', 'green', 'blue']
bars = ax2.bar(models, sizes, color=colors, alpha=0.7)
ax2.set_title('Model Size (MB)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Size (MB)')
for bar, size in zip(bars, sizes):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{size} MB', ha='center', va='bottom', fontweight='bold')

# Plot 3: Training Time Comparison
ax3 = axes[1, 0]
times = [vgg16_time, resnet50_time, mobilenet_time]
bars = ax3.bar(models, times, color=colors, alpha=0.7)
ax3.set_title('Training Time (5 epochs)', fontsize=14, fontweight='bold')
ax3.set_ylabel('Time (seconds)')
for bar, t in zip(bars, times):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             f'{t:.0f}s', ha='center', va='bottom', fontweight='bold')

# Plot 4: Final Accuracy Comparison
ax4 = axes[1, 1]
accuracies = [vgg16_acc * 100, resnet50_acc * 100, mobilenet_acc * 100]
bars = ax4.bar(models, accuracies, color=colors, alpha=0.7)
ax4.set_title('Final Validation Accuracy', fontsize=14, fontweight='bold')
ax4.set_ylabel('Accuracy (%)')
ax4.set_ylim([80, 100])
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{acc:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## Step 8: Decision Guide - Which Model to Use?

### Use VGG16 if:
- 📚 You're learning deep learning
- 🎓 Teaching/educational purposes
- 🔍 Need simple architecture to understand
- ⚠️ **NOT recommended for production**

---

### Use ResNet50 if: ⭐ DEFAULT CHOICE
- 🎯 You want best accuracy
- 💻 You have standard compute resources
- 🏢 Building production applications
- ⚖️ You want balanced size/speed/accuracy
- **→ RECOMMENDED for most projects!**

---

### Use MobileNetV2 if:
- 📱 Deploying to mobile devices
- ⚡ Need fast inference speed
- 💾 Limited storage/memory
- 🌐 Edge computing
- ✅ Can accept slightly lower accuracy (87% vs 91%)

---

### Quick Decision Tree:

```
START
  |
  ├─ Mobile/Edge deployment? ──YES──> MobileNetV2 📱
  │                         └─NO
  │                            |
  ├─ Learning/Teaching? ────────YES──> VGG16 📚
  │                         └─NO
  │                            |
  └─ Production application? ───YES──> ResNet50 ⭐
```

**Still unsure? → Use ResNet50!**

---

## 🎓 Summary: What You Learned

### Model Comparison:

| Feature | VGG16 | ResNet50 | MobileNetV2 |
|---------|-------|----------|-------------|
| **Year** | 2014 | 2015 | 2018 |
| **Size** | 528 MB | 98 MB | 14 MB |
| **Params** | 138M | 26M | 3.5M |
| **Accuracy** | ~89% | ~91% | ~87% |
| **Speed** | Slow | Medium | Fast |
| **Best For** | Learning | Default ⭐ | Mobile |

### Key Insights:
1. ✅ **ResNet50** = Best all-around choice
2. ✅ **MobileNetV2** = 10× smaller, 2× faster, only 4% accuracy loss
3. ✅ **VGG16** = Simple but outdated, good for learning
4. ✅ **Tradeoff**: Size ↔ Accuracy ↔ Speed

### Real-World Recommendations:
- **Startup prototype:** ResNet50 (fast development)
- **Mobile app:** MobileNetV2 (small size)
- **Cloud API:** ResNet50 (best accuracy)
- **IoT device:** MobileNetV2 (low memory)
- **Learning project:** VGG16 (simple architecture)

---

## ✅ Key Takeaways

- ✅ **Different models have different strengths**
- ✅ **ResNet50 is the default choice for most tasks**
- ✅ **MobileNetV2 is perfect for mobile/edge deployment**
- ✅ **VGG16 is good for learning but outdated**
- ✅ **Always consider: size, speed, accuracy, deployment target**

---

## 🎉 Congratulations!

You've completed all 4 transfer learning notebooks!

**What you mastered:**
1. ✅ **Notebook 01:** Why transfer learning is necessary
2. ✅ **Notebook 02:** Feature extraction (90% accuracy)
3. ✅ **Notebook 03:** Fine-tuning (93% accuracy)
4. ✅ **Notebook 04:** Model selection strategies

**You can now:**
- Apply transfer learning to any image dataset
- Choose the right pre-trained model
- Decide between feature extraction and fine-tuning
- Build production-ready image classifiers

---

## 🚀 Next Steps

**Apply your knowledge:**
1. Try different datasets (Cats vs Dogs, Food-101, etc.)
2. Experiment with other models (EfficientNet, InceptionV3)
3. Build a real application (medical, agriculture, wildlife)
4. Prepare for **FT2 test on Nov-14** 📝

**Practice questions:**
- "Given 2,000 medical images, which strategy would you use?"
- "Compare VGG16 and MobileNetV2 for mobile deployment"
- "Explain why fine-tuning requires lower learning rate"

---

**End of Notebook 04**

**Status:** ✅ All transfer learning notebooks complete!

**Achievement Unlocked:** 🏆 Transfer Learning Master

**Total time:** ~35-40 minutes

**Ready for:** Real-world projects! 🎉